In [1]:
# Loading pre-trained segmentation model, just run it by "Ctrl + Enter"

from stardist.models import StarDist2D 

# prints a list of available models 
StarDist2D.from_pretrained() 

# creates a pretrained model
model = StarDist2D.from_pretrained('2D_versatile_fluo')

from stardist.data import test_image_nuclei_2d
from stardist.plot import render_label
from csbdeep.utils import normalize
from matplotlib import image
import numpy as np
from tifffile import imsave

import matplotlib.pyplot as plt
import time
import numpy as np
import os
import cv2
from os import listdir


# Test if GPU is available
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

import tensorflow as tf

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

tf.test.is_gpu_available(cuda_only=True, min_cuda_compute_capability=(6,0))


There are 4 registered models for 'StarDist2D':

Name                  Alias(es)
────                  ─────────
'2D_versatile_fluo'   'Versatile (fluorescent nuclei)'
'2D_versatile_he'     'Versatile (H&E nuclei)'
'2D_paper_dsb2018'    'DSB 2018 (from StarDist 2D paper)'
'2D_demo'             None
Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18187075897435400635
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 2867814761121741734
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2203949467
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2195668724899029119
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1050, pci bus id:

True

In [38]:
# Pre-defined functions 
def getImgPrefix(myDir):
    imgPrefix = [];
    files = listdir(myDir)
    theName = files[0]
    str = theName.split('_frame')
    imgPrefix = str[0]
    if not imgPrefix:
        print('[Info] ' 'Pls make sure your images are with keywords xxx_framexxxxx_ROIx')  
    else:
        print('[Info] ' 'Folder ' + myDir + ' with image prefix: ' + imgPrefix)
        
    return imgPrefix   
    

def getSegmentationUsingStardist(frames_to_process, ROIs_to_process, path_raw, path_seg, image_prefix,remove_bg=False):
    
    for frame in (frames_to_process):    
        for ROI in (ROIs_to_process):   
            
            imgName = image_prefix + ("_frame%05d_ROI%03d.tif" % (frame, ROI))
#             imgName = "parts_ROI%03d.tif" % ROI
            
            imgName2 = path_raw + '\\' + imgName
            
            isImgExist = os.path.exists(imgName2)
        
            if not isImgExist:
                print('[Info] ' + imgName +' not generated yet ...')
                break
        
            print('[Info] ' + imgName +' is generated!'); time.sleep(1)
        
            img = image.imread(imgName2)
            print('[Info] ' + 'Segmenting ... ' + imgName)
            
            if remove_bg == True:
                filterSize = (30, 30)
                kernel = cv2.getStructuringElement(cv2.MORPH_RECT, filterSize)        
                img = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, kernel)
            
            #img = cv2.GaussianBlur(img, (21, 21), -1)
        
            t = time.time()
            
            imgHeight, imgWidth = img.shape; 
            labels, polys = model.predict_instances(normalize(img), n_tiles = (4,4))
            
            elapsed = time.time() - t; 
            centers = polys["points"]  # [y, x]
            temp = np.copy(centers[:, 0])
            centers[:, 0] = centers[:, 1]
            centers[:, 1] = temp
#             np.savetxt(path_seg + '\\' + image_prefix + "_frame%05d_ROI%03d.csv" % (frame, ROI), centers, delimiter=",")
            np.savetxt(path_seg + "\\ROI%03d.csv" %  ROI, centers, delimiter=",")

            imsave(path_seg + "\\" + imgName, np.uint16(labels))  
        
            print('[Info] ' '%5d cells found.' % centers.shape[0])
            print('[Info] ' + 'Segmentation and detected coordinates are saved into ' + path_seg)      

            
            print('[Info] ' 'Segmenting took %d seconds.' % (elapsed))
   
    

In [39]:
# Configure all file names and folder names

#path = r'D:\Data\Amke\test_olddata'
# path = r'D:\Data\Amke\AB009\step2_all_channels_20250319at133800'
# path = r"\\store.erasmusmc.nl\department\gene\chien_data\Lab\Data_and_Analysis\Amke Bernaerts\AB010_test_run"
# path = "D:\Data\Amke\AB012\step2 imaging all channels_20250507at121415"
# path = r"Z:\Data_and_Analysis\Amke Bernaerts\chapter4_thesis_image_analysis\segmentation\PARTS_ROIs"
path = r"Z:\Data_and_Analysis\Amke Bernaerts\chapter4_thesis_image_analysis\segmentation\UFO_ROIs"

path_460 = path + '\\460nm'
path_532 = path + '\\532nm'
path_637 = path + '\\637nm'

path_460_seg = path + '\\460nm_seg_bg_removed'
path_532_seg = path + '\\532nm_seg_bg_removed'
path_637_seg = path + '\\637nm_seg_bg_removed'

# path_results = path + '\\results_tagtest'
path_460results = path + '\\results_460nm_bgrem'
path_532results = path + '\\results_532nm_bgrem'
path_640results = path + '\\results_637nm_bgrem'

import os
if not os.path.isdir(path_460_seg):
    os.mkdir(path_460_seg)
if not os.path.isdir(path_532_seg):
    os.mkdir(path_532_seg)
if not os.path.isdir(path_637_seg):
    os.mkdir(path_637_seg)  
if not os.path.isdir(path_460results):
    os.mkdir(path_460results)   
if not os.path.isdir(path_532results):
    os.mkdir(path_532results)
if not os.path.isdir(path_640results):
    os.mkdir(path_640results)      
    

# Checking image prefix: if we are loading the right images from the right channel
image_prefix_460 = getImgPrefix(path_460)
image_prefix_532 = getImgPrefix(path_532)
image_prefix_637 = getImgPrefix(path_637)

[Info] Folder Z:\Data_and_Analysis\Amke Bernaerts\chapter4_thesis_image_analysis\segmentation\UFO_ROIs\460nm with image prefix: step2_all_channels_050B_1000ms
[Info] Folder Z:\Data_and_Analysis\Amke Bernaerts\chapter4_thesis_image_analysis\segmentation\UFO_ROIs\532nm with image prefix: step2_all_channels_100G_1000ms
[Info] Folder Z:\Data_and_Analysis\Amke Bernaerts\chapter4_thesis_image_analysis\segmentation\UFO_ROIs\637nm with image prefix: step2_all_channels_070R_1000ms


In [40]:
## define if try 1 image or do segmentation for all ROIs
try_one_img = False

if try_one_img == True:
    ROI = [10]
    ROIs_to_process = ROI
    ROIs_637 = ROI
    ROIs_460 = ROI
    ROIs_532 = ROI
else:
#     ROI = np.arange(0, 11, 1)
    ROI = [107,110,121]
    ROIs_to_process = ROI
    ROIs_637 = ROI
    ROIs_460 = ROI
    ROIs_532 = ROI
    

In [41]:
# To process 637nm - nuclei of all cells 

frames_to_process = [0]
# ROIs_to_process = np.arange(0, 253, 4)

getSegmentationUsingStardist(frames_to_process, ROIs_to_process, path_637, path_637_seg, image_prefix_637,remove_bg=True)


[Info] step2_all_channels_070R_1000ms_frame00000_ROI107.tif is generated!
[Info] Segmenting ... step2_all_channels_070R_1000ms_frame00000_ROI107.tif


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:06<00:00,  2.58it/s]


[Info] 23219 cells found.
[Info] Segmentation and detected coordinates are saved into Z:\Data_and_Analysis\Amke Bernaerts\chapter4_thesis_image_analysis\segmentation\UFO_ROIs\637nm_seg_bg_removed
[Info] Segmenting took 15 seconds.
[Info] step2_all_channels_070R_1000ms_frame00000_ROI110.tif is generated!
[Info] Segmenting ... step2_all_channels_070R_1000ms_frame00000_ROI110.tif


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:03<00:00,  4.19it/s]


[Info] 24540 cells found.
[Info] Segmentation and detected coordinates are saved into Z:\Data_and_Analysis\Amke Bernaerts\chapter4_thesis_image_analysis\segmentation\UFO_ROIs\637nm_seg_bg_removed
[Info] Segmenting took 13 seconds.
[Info] step2_all_channels_070R_1000ms_frame00000_ROI121.tif is generated!
[Info] Segmenting ... step2_all_channels_070R_1000ms_frame00000_ROI121.tif


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:03<00:00,  4.49it/s]


[Info] 17227 cells found.
[Info] Segmentation and detected coordinates are saved into Z:\Data_and_Analysis\Amke Bernaerts\chapter4_thesis_image_analysis\segmentation\UFO_ROIs\637nm_seg_bg_removed
[Info] Segmenting took 10 seconds.


In [42]:
# To process 532nm - immune cell signal

frames_to_process = [0]
# ROIs_to_process = np.arange(0, 11, 1)
# ROIs_to_process = [10]

getSegmentationUsingStardist(frames_to_process, ROIs_to_process, path_532, path_532_seg, image_prefix_532,remove_bg=True)

[Info] step2_all_channels_100G_1000ms_frame00000_ROI107.tif is generated!
[Info] Segmenting ... step2_all_channels_100G_1000ms_frame00000_ROI107.tif


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:03<00:00,  4.58it/s]


[Info] 13896 cells found.
[Info] Segmentation and detected coordinates are saved into Z:\Data_and_Analysis\Amke Bernaerts\chapter4_thesis_image_analysis\segmentation\UFO_ROIs\532nm_seg_bg_removed
[Info] Segmenting took 7 seconds.
[Info] step2_all_channels_100G_1000ms_frame00000_ROI110.tif is generated!
[Info] Segmenting ... step2_all_channels_100G_1000ms_frame00000_ROI110.tif


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:03<00:00,  4.36it/s]


[Info] 11430 cells found.
[Info] Segmentation and detected coordinates are saved into Z:\Data_and_Analysis\Amke Bernaerts\chapter4_thesis_image_analysis\segmentation\UFO_ROIs\532nm_seg_bg_removed
[Info] Segmenting took 7 seconds.
[Info] step2_all_channels_100G_1000ms_frame00000_ROI121.tif is generated!
[Info] Segmenting ... step2_all_channels_100G_1000ms_frame00000_ROI121.tif


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:03<00:00,  4.58it/s]


[Info] 10343 cells found.
[Info] Segmentation and detected coordinates are saved into Z:\Data_and_Analysis\Amke Bernaerts\chapter4_thesis_image_analysis\segmentation\UFO_ROIs\532nm_seg_bg_removed
[Info] Segmenting took 6 seconds.


In [43]:
# To process 460nm - tumor signal

frames_to_process = [0]
# ROIs_to_process =  np.arange(0, 11, 1)

getSegmentationUsingStardist(frames_to_process, ROIs_to_process, path_460, path_460_seg, image_prefix_460,remove_bg=True)

[Info] step2_all_channels_050B_1000ms_frame00000_ROI107.tif is generated!
[Info] Segmenting ... step2_all_channels_050B_1000ms_frame00000_ROI107.tif


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:03<00:00,  4.58it/s]


[Info] 13708 cells found.
[Info] Segmentation and detected coordinates are saved into Z:\Data_and_Analysis\Amke Bernaerts\chapter4_thesis_image_analysis\segmentation\UFO_ROIs\460nm_seg_bg_removed
[Info] Segmenting took 7 seconds.
[Info] step2_all_channels_050B_1000ms_frame00000_ROI110.tif is generated!
[Info] Segmenting ... step2_all_channels_050B_1000ms_frame00000_ROI110.tif


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:03<00:00,  4.52it/s]


[Info]  7891 cells found.
[Info] Segmentation and detected coordinates are saved into Z:\Data_and_Analysis\Amke Bernaerts\chapter4_thesis_image_analysis\segmentation\UFO_ROIs\460nm_seg_bg_removed
[Info] Segmenting took 6 seconds.
[Info] step2_all_channels_050B_1000ms_frame00000_ROI121.tif is generated!
[Info] Segmenting ... step2_all_channels_050B_1000ms_frame00000_ROI121.tif


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:03<00:00,  4.62it/s]


[Info]  2748 cells found.
[Info] Segmentation and detected coordinates are saved into Z:\Data_and_Analysis\Amke Bernaerts\chapter4_thesis_image_analysis\segmentation\UFO_ROIs\460nm_seg_bg_removed
[Info] Segmenting took 4 seconds.


In [45]:
from skimage.measure import label, regionprops, regionprops_table
from skimage import morphology
import csv
import numpy

# ROIs_637 = np.arange(0, 11, 1)
temp_imgs = []
frame = 0
# ROI_637 = 0

for ROI_637 in ROIs_637:

#     csvname = path_637_seg + '\\' + image_prefix_637 + ("_frame%05d_ROI%03d.csv" % (frame, ROI_637))
    csvname = path_637_seg + "\\ROI%03d.csv" % ROI_637
    print(csvname)
    
    coordinates_all_nuclei = np.genfromtxt(csvname, delimiter = ',')

#     temp_img = np.zeros((1164, 1164))
    temp_img = np.zeros((5120, 5120))
    
    for i in range(0, coordinates_all_nuclei.shape[0]):
        x = int(coordinates_all_nuclei[i, 0])
        y = int(coordinates_all_nuclei[i, 1])
        temp_img[y, x] = 1
    
    temp_imgs.append(temp_img)

Z:\Data_and_Analysis\Amke Bernaerts\chapter4_thesis_image_analysis\segmentation\UFO_ROIs\637nm_seg_bg_removed\ROI107.csv
Z:\Data_and_Analysis\Amke Bernaerts\chapter4_thesis_image_analysis\segmentation\UFO_ROIs\637nm_seg_bg_removed\ROI110.csv
Z:\Data_and_Analysis\Amke Bernaerts\chapter4_thesis_image_analysis\segmentation\UFO_ROIs\637nm_seg_bg_removed\ROI121.csv


In [49]:
##now this dilates both tumor and immune channel

# Final step: organize all segmented channels  

# To load immune and tumor cell channels, and try to remove overlapping regions, for avoiding tagging cotamination

dilate_size = 20
dilate_size_smaller = 10

# to load immune cell channel (532nm)
# ROIs_532 = np.arange(0, 11, 1)
seg_532s = []
seg_532s_undilated = []
seg_460s_undilated = []

for ROI_532 in ROIs_532:
    seg_532 = image.imread(path_532_seg + '\\' + image_prefix_532 + ("_frame%05d_ROI%03d.tif" % (frame, ROI_532)))
#     seg_532 = image.imread(path_532_seg + "\\parts_ROI%03d.tif" % ROI_532)
    
    #small dilation
    kernel = np.ones((dilate_size_smaller, dilate_size_smaller), np.uint16) 
    seg_532_di_small = cv2.dilate(seg_532, kernel, iterations=1) 
    seg_532_small = seg_532_di_small
    seg_532s_undilated.append(seg_532_small)

    # dilate the seg_532
    kernel = np.ones((dilate_size, dilate_size), np.uint16) 
    seg_532_di = cv2.dilate(seg_532, kernel, iterations=1) 
    seg_532 = seg_532_di

    seg_532s.append(seg_532)

# to load cancer cell channel (460nm)
# ROIs_460 = np.arange(0, 11, 1)
seg_460s = []

for ROI_460 in ROIs_460:
#     ROI_460 = 77
    # print(ROI_460)
    seg_460 = image.imread(path_460_seg + '\\' + image_prefix_460 + ("_frame%05d_ROI%03d.tif" % (frame, ROI_460)))
#     seg_460 = image.imread(path_460_seg + "\\parts_ROI%03d.tif" % ROI_460)
    
    #small dilation
    kernel = np.ones((dilate_size_smaller, dilate_size_smaller), np.uint16) 
    seg_460_di_small = cv2.dilate(seg_460, kernel, iterations=1) 
    seg_460_small = seg_460_di_small
    seg_460s_undilated.append(seg_460_small)
    
    seg_460s_undilated.append(seg_460)

    # dilate the seg_532
    kernel = np.ones((dilate_size, dilate_size), np.uint16) 
    seg_460_di = cv2.dilate(seg_460, kernel, iterations=1) 
    seg_460 = seg_460_di

    seg_460s.append(seg_460)

# to remove overlapping regions
temp_seg = (seg_532_di > 0) + (seg_460_di > 0)
temp_seg = (temp_seg == 2)

seg_532 = seg_532 - temp_seg
seg_460 = seg_460 - temp_seg


In [ ]:
# original code of the function below was:
# #check if immune cells overlap with tumor cells, if yes, don't use for tagging
# for i in range(len(ROIs_532)):
#     img_immune = seg_532s_undilated[i]
#     img_immune_binary = img_immune>0
#     img_tumor = seg_460s_undilated[i]
#     img_tumor_binary = img_tumor>0
#     intersect = img_tumor_binary*img_immune_binary
#     plt.imshow(img_tumor_binary)
#     plt.title('tumor')
#     plt.show()
#     plt.imshow(img_immune_binary)
#     plt.title('immune')
#     plt.show()
#     plt.imshow(intersect)
#     plt.title('intersect')
#     plt.show()
#     only_immune_signal = np.zeros([len(intersect),len(intersect)])
#     for k in range(len(intersect)):
#         for j in range(len(intersect)):
#             if img_immune_binary[k,j] == True and intersect[k,j] == False:
#                 only_immune_signal[k,j] = 1
            
# #     only_immune_signal = img_immune_binary - intersect
#     only_immune_signals.append(only_immune_signal)
#     plt.imshow(only_immune_signal)
#     plt.title('difference')
#     plt.show()
    
#     ROI = ROIs_532[i]
#     coord_file = path_532_seg + "\\ROI%03d.csv" % ROI
#     print(coord_file)
#     seg_532_coord = np.genfromtxt(coord_file, delimiter = ',')
#     print('all immune coords',len(seg_532_coord))
#     for c in range(len(seg_532_coord)):
#         x = int(seg_532_coord[c][0])
#         y = int(seg_532_coord[c][1])
#         if only_immune_signal[y,x] > 0:  #still ask li if this is correct, number makes most sense but why y,x?
#             only_immune_coords.append([x,y])
    
# print('only immune coords',len(only_immune_coords))
# # np.savetxt("X:\Amke Bernaerts\AB010_test_run\only_immune_coords.csv",only_immune_coords,delimiter=",")


In [30]:
def find_overlapping_tumor_immune_cells(path1,ROIs,seg1_undilated,seg2_undilated,plot=False,save=False):
    #for immune coordinates: seg1 = seg_532s_undilated and seg2 = seg_460s_undilated
    # and path1 = path_532_seg
    
    final_masks = []
    final_coords = []
    
    for i in range(len(ROIs)):
        img_immune = seg1_undilated[i]
        img_immune_binary = img_immune>0
        img_tumor = seg2_undilated[i]
        img_tumor_binary = img_tumor>0
        intersect = img_tumor_binary*img_immune_binary
        if plot == True:
            plt.imshow(img_tumor_binary)
            plt.title('tumor')
            plt.show()
            plt.imshow(img_immune_binary)
            plt.title('immune')
            plt.show()
            plt.imshow(intersect)
            plt.title('intersect')
            plt.show()

        only_immune_signal = np.zeros([len(intersect),len(intersect)])
        for k in range(len(intersect)):
            for j in range(len(intersect)):
                if img_immune_binary[k,j] == True and intersect[k,j] == False:
                    only_immune_signal[k,j] = 1

    #     only_immune_signal = img_immune_binary - intersect
        final_masks.append(only_immune_signal)
        if plot == True:
            plt.imshow(only_immune_signal)
            plt.title('difference')
            plt.show()

        ROI = ROIs[i]
        coord_file = path1 + "\\ROI%03d.csv" % ROI
#         print(coord_file)
        seg_532_coord = np.genfromtxt(coord_file, delimiter = ',')
        print('number of coordinates before removing overlap is ',len(seg_532_coord))
        for c in range(len(seg_532_coord)):
            x = int(seg_532_coord[c][0])
            y = int(seg_532_coord[c][1])
            if only_immune_signal[y,x] > 0:  #still ask li if this is correct, number makes most sense but why y,x?
                final_coords.append([x,y])
    
    print('number of coordinates after removing overlap is ',len(final_coords))
    if save == True:
        np.savetxt("X:\Amke Bernaerts\AB010_test_run\only_immune_coords.csv",final_coords,delimiter=",")
    
    return final_masks

In [50]:
# to get Galvo tagging coordinates from 532nm
# summary of what this code does:
# checks if immune cells don't overlap with tumor cells (to get rid of wrongly segmented immune cells)
# check if the immune cell segmentation overlaps with a nucleus, only then keep tagging coordinate

from glob import glob

parts_pixelsize = 1.376 #um/pixel
ufo_pixelsize = 0.31295 #um/pixel
scaling_factor = parts_pixelsize/ufo_pixelsize
boundary = 100 #only tag cells that are at least this amount away from the edge (tagging in middle is more accurate)
img_size = 5120 #parts image size=1164

# ROIs_532 = np.arange(0, 11, 1)
# ROIs_532 = [10]


# #check if immune cells overlap with tumor cells, if yes, don't use for tagging
# final_masks = find_overlapping_tumor_immune_cells(path_532_seg,ROIs_532,seg_532s_undilated,seg_460s_undilated,plot=True)
# print(final_masks)
# check overlap with nuclei segmentation    
for i in range(len(temp_imgs)):
    temp_img = temp_imgs[i]
    seg_532 = seg_532s[i]
    ROI_532 = ROIs_532[i]
    
    
    img = temp_img*np.array(seg_532)
#     img = temp_img*np.array(final_masks[i])
    
    #imsave(path + '\\' 'testa.tif', img);
    
    L = label(img > 0)
    regions = regionprops(L)  #ask li about these two lines >> it only finds 1, what went wrong?
    
    coordinates_532 = np.array([[0, 0, 0, 0]])
    coordinates_532_ufo = np.array([[0, 0, 0, 0]])
    counter = 0
    for props in regions:
        y0, x0 = props.centroid
        if y0 > boundary and y0 < (img_size-boundary) and x0 > boundary and x0 < (img_size-boundary):
#             if [x0,y0] in only_immune_coords:
#             if x0 in only_immune_coords_x: #and y0 in only_immune_coords_y[np.where(only_immune_coords_x==x0)]:
#                 indices = np.where(only_immune_coords_x==x0)
# #                 print('indices',indices)
#                 y_coords_at_x = []
#                 for index in indices[0]:
# #                     print(index)
#                     y_coords_at_x.append(only_immune_coords_y[index])
#                 if y0 in y_coords_at_x:
#             print(x0,y0)
            new_row = np.array([counter, int(x0), int(y0), 2000])
            #print(new_row)
            coordinates_532 = np.append(coordinates_532, [new_row], axis=0)
            new_row_ufo = np.array([counter,int(x0*scaling_factor),int(y0*scaling_factor),2000])
            coordinates_532_ufo = np.append(coordinates_532_ufo, [new_row_ufo], axis=0)
            counter += 1 
        
    N = 10000
    
    coordinates_532_tag = coordinates_532[1:min(N, coordinates_532.shape[0]), 1:4]
    coordinates_532_imageJ = coordinates_532[1:min(N, coordinates_532.shape[0]), 0:3]
    
#     coordinates_532_tag_ufo = (parts_pixelsize/ufo_pixelsize)*coordinates_532[1:min(N, coordinates_532.shape[0]), 1:4]
#     coordinates_532_imageJ_ufo = (parts_pixelsize/ufo_pixelsize)*coordinates_532[1:min(N, coordinates_532.shape[0]), 0:3]
    coordinates_532_tag_ufo = coordinates_532_ufo[1:min(N, coordinates_532_ufo.shape[0]), 1:4]
    coordinates_532_imageJ_ufo = coordinates_532_ufo[1:min(N, coordinates_532_ufo.shape[0]), 0:3]
    
    print(np.max(coordinates_532_tag_ufo))
    print(np.shape(coordinates_532_tag_ufo))
#     np.savetxt(path_532results + '\\immune' + "_frame%05d_ROI%03d_CS%05d.csv" % (frame, ROI_532, min(N, coordinates_532_tag.shape[0])), coordinates_532_tag, delimiter=",")    
    np.savetxt(path_532results + '\\immune' + "_frame%05d_ROI%03d_ImageJ.csv" % (frame, ROI_532), coordinates_532_imageJ, delimiter=",") 

22063
(9808, 3)
22063
(9999, 3)
22063
(8239, 3)


In [36]:
def get_overlap(img1,img2): # task: 'intersect' or 'setdiff'
        #img1 = tumor dilated
        #img2 - immune dilated
        
    intersect = img1*img2
    img = img2-intersect
    return intersect
            

In [52]:
# to get Galvo tagging coordinates from 460nm
# ROIs_460 = np.arange(0, 11, 1)
ROIs_460 = [107,110,121]

# final_masks = find_overlapping_tumor_immune_cells(path_460_seg,ROIs_460,seg_460s_undilated,seg_532s_undilated,plot=False)

for i in range(len(temp_imgs)):
    temp_img = temp_imgs[i]
    seg_460 = seg_460s[i]
    ROI_460 = ROIs_460[i]
    

    img = temp_img*np.array(seg_460)
#     img = temp_img*np.array(final_masks[i])

    #imsave(path + '\\' 'testb.tif', img);

    L = label(img > 0)
    regions = regionprops(L)

    coordinates_460 = np.array([[0, 0, 0, 0]])
    coordinates_460_ufo = np.array([[0, 0, 0, 0]])
    counter = 0
    for props in regions:
        y0, x0 = props.centroid
        if y0 > boundary and y0 < (img_size-boundary) and x0 > boundary and x0 < (img_size-boundary):
            new_row = np.array([counter, x0, y0, 2000])
            #print(new_row)
            coordinates_460 = np.append(coordinates_460, [new_row], axis=0)
            new_row_ufo = np.array([counter,int(x0*scaling_factor),int(y0*scaling_factor),2000])
            coordinates_460_ufo = np.append(coordinates_460_ufo, [new_row_ufo], axis=0)
            counter += 1

    N = 10000

    coordinates_460_tag = coordinates_460[1:min(N,coordinates_460.shape[0]), 1:4]
    coordinates_460_imageJ = coordinates_460[1:min(N,coordinates_460.shape[0]), 0:3]
    
    coordinates_460_tag_ufo = coordinates_460_ufo[1:min(N, coordinates_460_ufo.shape[0]), 1:4]
    coordinates_460_imageJ_ufo = coordinates_460_ufo[1:min(N, coordinates_460_ufo.shape[0]), 0:3]
    
#     np.savetxt(path_460results + '\\tumor' + "_frame%05d_ROI%03d_CS%05d.csv" % (frame, ROI_460, min(N, coordinates_460_tag.shape[0])), coordinates_460_tag, delimiter=",") 
    np.savetxt(path_460results + '\\tumor' + "_frame%05d_ROI%03d_ImageJ.csv" % (frame, ROI_460), coordinates_460_imageJ, delimiter=",") 



In [ ]:
# to get Galvo tagging coordinates from 460nm - adjusted
ROIs_460 = np.arange(0, 11, 1)

ufo_path = r"\\store.erasmusmc.nl\department\gene\chien_data\Lab\Data_and_Analysis\Amke Bernaerts\AB009_insituFUNseq_washing_optimization\Raw data\AB009\step2_imaging_and_segmentation_20250319at133800\460nm_renamed"
ufo_files = glob(ufo_path+"\\*.tif")
ufo_files.sort()

for i in range(len(temp_imgs)):
    ufo_img = cv2.imread(ufo_files[i],cv2.IMREAD_UNCHANGED)
    
    temp_img = temp_imgs[i]
    seg_460 = seg_460s[i]
    ROI_460 = ROIs_460[i]

    img = temp_img*np.array(seg_460)

    #imsave(path + '\\' 'testb.tif', img);

    L = label(img > 0)
    regions = regionprops(L)

    coordinates_460 = np.array([[0, 0, 0, 0]])
    counter = 0
    for props in regions:
        y0, x0 = props.centroid
        ## here: only store the coordinates in the center of the image (e.g. 200 pixels from edge)
        new_row = np.array([counter, x0, y0, 2000])
        #print(new_row)
        coordinates_460 = np.append(coordinates_460, [new_row], axis=0)
        counter += 1

    N = 200

    coordinates_460_tag = coordinates_460[1:min(N,coordinates_460.shape[0]), 1:4]
    coordinates_460_imageJ = coordinates_460[1:min(N,coordinates_460.shape[0]), 0:3]
    np.savetxt(path_460results + '\\tumor' + "_frame%05d_ROI%03d_CS%05d.csv" % (frame, ROI_460, min(N, coordinates_460_tag.shape[0])), coordinates_460_tag, delimiter=",") 
    np.savetxt(path_460results + '\\tumor' + "_frame%05d_ROI%03d_ImageJ.csv" % (frame, ROI_460), coordinates_460_imageJ, delimiter=",") 


In [17]:
print(coordinates_460_tag.shape)

(72, 3)


In [73]:
# If you want to do DMD tagging

# to get DMD tagging mask for 532nm, using nuclei-signal * seg-532
ROI_637 = 33
seg_637 = image.imread(path_637_seg + '\\' + image_prefix_637 + ("_frame%05d_ROI%03d.tif" % (frame, ROI_637)))

DMD_mask_532 = (seg_532 > 0)*(seg_637 >0)
DMD_mask_532 = morphology.remove_small_objects(DMD_mask_532, min_size=100, connectivity=4)
imsave(path_results + '\\DMD_mask_immune' + "_frame%05d_ROI%03d.tif" % (frame, ROI_532), DMD_mask_532);

DMD_mask_460 = (seg_460 > 0)*(seg_637 >0)
DMD_mask_460 = morphology.remove_small_objects(DMD_mask_460, min_size=100, connectivity=4)
imsave(path_results + '\\DMD_mask_tumor' + "_frame%05d_ROI%03d.tif" % (frame, ROI_460), DMD_mask_460);

In [32]:
# Get non-tumor regions

# to get Galvo tagging coordinates from 460nm

csvname = path_637_seg + '\\' + image_prefix_637 + ("_frame%05d_ROI%03d.csv" % (frame, ROI_637))
print(csvname)
                                    
# to load cancer cell channel (460nm)
ROI_460 = 77
seg_460 = image.imread(path_460_seg + '\\' + image_prefix_460 + ("_frame%05d_ROI%03d.tif" % (frame, ROI_460)))

# dilate the seg_532
dilate_size = 200

kernel = np.ones((dilate_size, dilate_size), np.uint16) 
seg_460_di = cv2.dilate(seg_460, kernel, iterations=1) 
seg_460 = seg_460_di > 0   

#imsave(path + '\\' 'testtest.tif', seg_460);

coordinates_all_nuclei = np.genfromtxt(csvname, delimiter = ',')

coordinates_460 = np.array([[0, 0, 0, 0]])
counter = 0

for i in range(0, coordinates_all_nuclei.shape[0]):
    x = int(coordinates_all_nuclei[i, 0])
    y = int(coordinates_all_nuclei[i, 1])
    new_row = np.array([counter, x, y, 2000])
    
    if seg_460[y, x] > 0:
        continue
        
    coordinates_460 = np.append(coordinates_460, [new_row], axis=0)
    counter += 1 
        
N = 200

coordinates_460_tag = coordinates_460[1:min(N,coordinates_460.shape[0]), 1:4]
coordinates_460_imageJ = coordinates_460[1:min(N,coordinates_460.shape[0]), 0:3]
np.savetxt(path_results + '\\nontumor' + "_frame%05d_ROI%03d_CS%05d.csv" % (frame, ROI_460, min(N, coordinates_460.shape[0])), coordinates_460_tag, delimiter=",") 
np.savetxt(path_results + '\\nontumor' + "_frame%05d_ROI%03d_ImageJ.csv" % (frame, ROI_460), coordinates_460_imageJ, delimiter=",") 

D:\Data\Amke\AB009\step2_all_channels_20250319at133800\637nm_seg\step2_all_channels_070R_1000ms_frame00000_ROI076.csv
